In [1]:
import panel as pn
import os
import numpy as np
import bokeh

from bokeh.layouts import column, row, Spacer
from bokeh.models import CustomJS, Slider, RadioButtonGroup, TextInput, Button, MultiChoice
from bokeh.models import BoxAnnotation, PreText, Range1d, LinearAxis, Span, HoverTool, DataTable, TableColumn
from bokeh.events import SelectionGeometry
from bokeh.plotting import ColumnDataSource, figure, show

!python --version
!which python

Python 3.8.8
/home/jhoskins/anaconda3/bin/python


In [2]:
import sys
sys.path.append('/home/jhoskins/anaconda3/lib/python3.8/site-packages')

In [3]:
print(sys.path)

['/home/jhoskins/Development/benchmarking', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/jhoskins/.local/lib/python3.9/site-packages', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.9/dist-packages', '/home/jhoskins/.local/lib/python3.9/site-packages/IPython/extensions', '/home/jhoskins/.ipython', '/home/jhoskins/anaconda3/lib/python3.8/site-packages']


In [ ]:
!/home/jhoskins/anaconda3/bin/python -m pip install casatasks==6.2.0.124
!/home/jhoskins/anaconda3/bin/python -m pip install casatools==6.2.0.124
!/home/jhoskins/anaconda3/bin/python -m pip install casatestutils==6.2.0.124

In [ ]:
import casatasks
import casatools

In [8]:
import yaml

class TCleanOptions:
    def __init__(self, config_file:str):
        self.config_file: str = config_file
        
        self.vis: str 
        self.imagename: str 
        self.imsize: int 
        self.cell: str 
        self.specmode: str
        self.interpolation: str
        self.nchan: int
        self.start: str
        self.width: str
        self.pblimit: float
        self.deconvolver: str
        self.niter: int
        self.cyclefactor: int
        self.scales: list
        self.interactive: int
    
    def read_configuration(self)->None:
        with open(self.config_file) as file:
            config = yaml.full_load(file)
        
        self.vis = config['tclean-first-cycle']['vis']
        self.imagename = config['tclean-first-cycle']['imagename']
        self.imsize = config['tclean-first-cycle']['imsize']
        self.cell = config['tclean-first-cycle']['cell']
        self.specmode = config['tclean-first-cycle']['specmode']
        self.interpolation = config['tclean-first-cycle']['interpolation']
        self.nchan = config['tclean-first-cycle']['nchan']
        self.start = config['tclean-first-cycle']['start']
        self.width = config['tclean-first-cycle']['width']
        self.pblimit = config['tclean-first-cycle']['pblimit']
        self.deconvolver = config['tclean-first-cycle']['deconvolver']
        self.niter = config['tclean-first-cycle']['niter']
        self.cyclefactor = config['tclean-first-cycle']['cyclefactor']
        self.scales = config['tclean-first-cycle']['scales']


In [9]:
options = TCleanOptions('stakeholder_tools/config/tclean.yaml')
options.read_configuration()

In [10]:
pn.extension()

In [44]:
class TCleanPanel():
    
    def __init__(self):
        OPTIONS = [('I', 'I'), ('Q', 'Q'), ('U', 'U'), ('V', 'V')]
        
        self.file = pn.widgets.FileSelector('~', name="File Selector")
        
        self.vis = pn.widgets.TextInput(
            value="select ms file ...", 
            name="Vis")

        self.imagename = TextInput(
            value="output file name ...", 
            title="Imagename")
        
        self.imsize = pn.widgets.IntInput(
            name='imsize', 
            value=512, 
            step=1, 
            start=1, 
            end=1000)

        self.cell = pn.widgets.TextInput(
            value="8.0arcsec", 
            name="Cell")

        self.specmode = pn.widgets.TextInput(
            value="cube", 
            name="Specmode")
        
        self.interpolation = pn.widgets.TextInput(
            value="nearest", 
            name="Interpolation")
        
        self.nchan = pn.widgets.IntInput(
            name='nchan', 
            value=5, 
            step=1, 
            start=1, 
            end=500)

        self.start = pn.widgets.TextInput(
            value="1.0GHz", 
            name="Start")

        self.width = pn.widgets.TextInput(
            value="0.2GHz", 
            name="Width")
        
        self.pblimit = pn.widgets.FloatInput(
            name='pblimit', 
            value=-0.00001, 
            step=0.000001, 
            start=-0.000001, 
            end=-0.0001)

        self.deconvolver = pn.widgets.TextInput(
            value="hogbom", 
            name="Deconvolver")
        
        self.niter = pn.widgets.IntInput(
            name='niter', 
            value=5, 
            step=1, 
            start=1, 
            end=5000)
        
        self.cyclefactor = pn.widgets.IntInput(
            name='cyclefactor', 
            value=1, 
            step=1, 
            start=1, 
            end=500)
        
        self.interactive = pn.widgets.IntInput(
            name='interactive', 
            value=0, 
            step=1, 
            start=0, 
            end=1)

    
    # ------
    
        self.play_button = pn.widgets.Button(
            name="Play", 
            button_type="success",  
            margin=(5,1,5,1)) 
        #icon=SVGIcon(icon_name="play"))
    
        self.layout = pn.Row(
            pn.Accordion(self.file),
            pn.Column(
                self.vis,
                self.imagename,
                self.imsize,
                self.cell,
                self.specmode,
                self.interpolation,
                self.nchan,
                self.start,
                self.width,
                self.pblimit,
                self.deconvolver,
                self.niter,
                self.cyclefactor,
                self.interactive,
                self.play_button    
            )
        )

In [45]:
tclean = TCleanPanel()

In [46]:
tclean.layout

Row
    [0] Accordion
        [0] FileSelector(directory='/home/jhoskins', name='File Selector')
    [1] Column
        [0] TextInput(name='Vis', value='select ms file ...')
        [1] Bokeh(TextInput)
        [2] IntInput(end=1000, name='imsize', start=1, value=512)
        [3] TextInput(name='Cell', value='8.0arcsec')
        [4] TextInput(name='Specmode', value='cube')
        [5] TextInput(name='Interpolation', value='nearest')
        [6] IntInput(end=500, name='nchan', start=1, value=5)
        [7] TextInput(name='Start', value='1.0GHz')
        [8] TextInput(name='Width', value='0.2GHz')
        [9] FloatInput(end=-0.0001, name='pblimit', start=-1e-06, step=1e-06, value=-1e-05)
        [10] TextInput(name='Deconvolver', value='hogbom')
        [11] IntInput(end=5000, name='niter', start=1, value=5)
        [12] IntInput(end=500, name='cyclefactor', start=1, value=1)
        [13] IntInput(end=1, name='interactive', start=0)
        [14] Button(button_type='success', margin=(5, 1, 5, 1), name='Play')

In [35]:
tclean.file.value

['/home/jhoskins/Development/benchmarking/data/E2E6.1.00034.S_tclean.ms',
 '/home/jhoskins/Development/benchmarking/data/sim_data_VLA.ms']

In [21]:
tclean.interpolation.param.watch(print, 'value')

Watcher(inst=TextInput(name='Interpolation', value='farthest', value_input='farthest'), cls=<class 'panel.widgets.input.TextInput'>, fn=<built-in function print>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False)

In [22]:
tclean.interpolation.value = 'nearest'

Event(what='value', name='value', obj=TextInput(name='Interpolation', value='nearest', value_input='farthest'), cls=TextInput(name='Interpolation', value='nearest', value_input='farthest'), old='farthest', new='nearest', type='changed')
Event(what='value', name='value', obj=TextInput(name='Interpolation', value='nearest', value_input='farthest'), cls=TextInput(name='Interpolation', value='nearest', value_input='farthest'), old='farthest', new='nearest', type='changed')
Event(what='value', name='value', obj=TextInput(name='Interpolation', value='nearest', value_input='farthest'), cls=TextInput(name='Interpolation', value='nearest', value_input='farthest'), old='farthest', new='nearest', type='changed')


In [24]:
tclean.interpolation.value

'boo'